# Introduction
Insallling Some Library from from Kaggle

In [2]:
# !pip download tsflex seglearn pycatch22

In [3]:
# Install tsflex and seglearn
# !pip install tsflex --no-index --find-links=time_series_tools
# !pip install seglearn --no-index --find-links=time_series_tools

Download and import necessary library to run the project

In [4]:
import numpy as np
import pandas as pd
from sklearn import *
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path

# Grab important files
Data Pre-procesing

In [5]:
root = '.'

train = glob.glob(path.join(root, 'train/**/**'))
test = glob.glob(path.join(root, 'test/**/**'))

subjects = pd.read_csv(path.join(root, 'subjects.csv'))
tasks = pd.read_csv(path.join(root, 'tasks.csv'))
events = pd.read_csv(path.join(root, 'events.csv'))

tdcsfog_metadata = pd.read_csv(path.join(root, 'tdcsfog_metadata.csv'))
defog_metadata = pd.read_csv(path.join(root, 'defog_metadata.csv')) 

tdcsfog_metadata['Module'] = 'tdcsfog'
defog_metadata['Module'] = 'defog'

full_metadata = pd.concat([tdcsfog_metadata, defog_metadata])

In [6]:
# View Subject head
subjects.head()

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,00f674,2.0,63,M,27.0,43.0,49.0,24
1,00f674,1.0,63,M,27.0,31.0,30.0,26
2,02bc69,NaN,69,M,4.0,21.0,NaN,22
3,040587,2.0,75,M,26.0,52.0,69.0,21
4,040587,1.0,75,M,26.0,47.0,75.0,24


In [7]:
#View Specific Subject
subjects[subjects['Subject'] == 'fe5d84']

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ


In [8]:
subjects.loc[subjects['Subject'] == 'fe5d84', 'Sex'] = 'F'
subjects[subjects['Subject'] == 'fe5d84']

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ


In [9]:
# View Subject Information
subjects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Subject       173 non-null    object 
 1   Visit         111 non-null    float64
 2   Age           173 non-null    int64  
 3   Sex           173 non-null    object 
 4   YearsSinceDx  173 non-null    float64
 5   UPDRSIII_On   172 non-null    float64
 6   UPDRSIII_Off  132 non-null    float64
 7   NFOGQ         173 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 10.9+ KB


In [10]:
subjects['Sex'] = subjects['Sex'].factorize()[0]
seed = 3
cluster_size = 8

In [11]:
subjects = subjects.fillna(0).groupby('Subject').median()
subjects['s_group'] = cluster.KMeans(n_clusters = cluster_size, random_state = seed).fit_predict(subjects[subjects.columns[1:]])
new_names = {'Visit':'s_visit','Age':'s_age','YearsSinceDx':'s_years','UPDRSIII_On':'s_on','UPDRSIII_Off':'s_off','NFOGQ':'s_NFOGQ', 'Sex': 's_sex'}
subjects = subjects.rename(columns = new_names)
subjects

c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
Subject,,,,,,,,
00f674,1.5,63.0,0.0,27.0,37.0,39.5,25.0,0
02bc69,0.0,69.0,0.0,4.0,21.0,0.0,22.0,3
040587,1.5,75.0,0.0,26.0,49.5,72.0,22.5,4
056372,2.0,69.0,0.0,13.0,44.0,50.0,22.0,6
07285e,0.0,58.0,0.0,1.0,18.0,26.0,10.0,2
...,...,...,...,...,...,...,...,...
f686f0,0.0,61.0,0.0,7.0,44.0,0.0,24.0,1
f80507,1.0,57.0,0.0,2.0,12.0,0.0,0.0,3
fa8764,0.0,60.0,1.0,7.0,30.0,0.0,19.0,1


In [12]:
# View Task Head
tasks.head()

,Id,Begin,End,Task
0,02ab235146,10.00,190.48,Rest1
1,02ab235146,211.24,271.56,Rest2
2,02ab235146,505.88,522.40,4MW
3,02ab235146,577.96,594.64,4MW-C
4,02ab235146,701.32,715.28,MB1


In [13]:
# View Task Information
tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      2817 non-null   object 
 1   Begin   2817 non-null   float64
 2   End     2817 non-null   float64
 3   Task    2817 non-null   object 
dtypes: float64(2), object(2)
memory usage: 88.2+ KB


In [14]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_group'] = cluster.KMeans(n_clusters = cluster_size, random_state = seed).fit_predict(tasks[tasks.columns[1:]])

c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [15]:
tasks

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,t_group
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,0
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,5
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,1
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,5
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,1
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,1
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,1
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,5


In [16]:
tdcsfog_metadata.head()

,Id,Subject,Visit,Test,Medication,Module
0,003f117e14,4dc2f8,3,2,on,tdcsfog
1,009ee11563,f62eec,4,2,on,tdcsfog
2,011322847a,231c3b,2,2,on,tdcsfog
3,01d0fe7266,231c3b,2,1,off,tdcsfog
4,024418ba39,fa8764,19,3,on,tdcsfog


In [17]:
defog_metadata.head()

,Id,Subject,Visit,Medication,Module
0,02ab235146,e1f62e,2,on,defog
1,02ea782681,ae2d35,2,on,defog
2,06414383cf,8c1f5e,2,off,defog
3,092b4c1819,2874c5,1,off,defog
4,0a900ed8a2,0e3d49,2,on,defog


In [18]:
# merge the subjects with the metadata
metadata_w_subjects = full_metadata.merge(subjects, how='left', on='Subject').copy()
features = metadata_w_subjects.columns
metadata_w_subjects

,Id,Subject,Visit,Test,Medication,Module,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
0,003f117e14,4dc2f8,3,2.0,on,tdcsfog,0.0,68.0,1.0,9.0,17.0,15.0,15.0,2
1,009ee11563,f62eec,4,2.0,on,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0,1
2,011322847a,231c3b,2,2.0,on,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
3,01d0fe7266,231c3b,2,1.0,off,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
4,024418ba39,fa8764,19,3.0,on,tdcsfog,0.0,60.0,1.0,7.0,30.0,0.0,19.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,1a778d,1,NaN,off,defog,1.5,65.0,0.0,7.0,50.0,59.5,24.5,6
966,f40e8c6ebe,575c60,1,NaN,off,defog,1.0,28.0,0.0,4.0,54.0,50.0,25.0,0
967,f8ddbdd98d,107712,1,NaN,on,defog,1.0,82.0,1.0,11.0,38.0,42.0,21.0,5
968,f9efef91fb,5d9cae,2,NaN,off,defog,1.5,72.0,0.5,14.0,22.5,39.0,16.0,5


In [19]:
features

Index(['Id', 'Subject', 'Visit', 'Test', 'Medication', 'Module', 's_visit',
       's_age', 's_sex', 's_years', 's_on', 's_off', 's_NFOGQ', 's_group'],
      dtype='object')

In [20]:
metadata_w_subjects['Medication'] = metadata_w_subjects['Medication'].factorize()[0]
metadata_w_subjects

,Id,Subject,Visit,Test,Medication,Module,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog,0.0,68.0,1.0,9.0,17.0,15.0,15.0,2
1,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,0.0,10.0,42.0,0.0,24.0,1
2,011322847a,231c3b,2,2.0,0,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
3,01d0fe7266,231c3b,2,1.0,1,tdcsfog,0.0,67.0,0.0,12.0,27.0,28.0,19.0,2
4,024418ba39,fa8764,19,3.0,0,tdcsfog,0.0,60.0,1.0,7.0,30.0,0.0,19.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,1a778d,1,NaN,1,defog,1.5,65.0,0.0,7.0,50.0,59.5,24.5,6
966,f40e8c6ebe,575c60,1,NaN,1,defog,1.0,28.0,0.0,4.0,54.0,50.0,25.0,0
967,f8ddbdd98d,107712,1,NaN,0,defog,1.0,82.0,1.0,11.0,38.0,42.0,21.0,5
968,f9efef91fb,5d9cae,2,NaN,1,defog,1.5,72.0,0.5,14.0,22.5,39.0,16.0,5


In [21]:
#Display Information
display(tasks)
display(subjects)

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,t_group
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,0
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,5
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,1
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,5
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,1
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,1
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,1
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,5


,s_visit,s_age,s_sex,s_years,s_on,s_off,s_NFOGQ,s_group
Subject,,,,,,,,
00f674,1.5,63.0,0.0,27.0,37.0,39.5,25.0,0
02bc69,0.0,69.0,0.0,4.0,21.0,0.0,22.0,3
040587,1.5,75.0,0.0,26.0,49.5,72.0,22.5,4
056372,2.0,69.0,0.0,13.0,44.0,50.0,22.0,6
07285e,0.0,58.0,0.0,1.0,18.0,26.0,10.0,2
...,...,...,...,...,...,...,...,...
f686f0,0.0,61.0,0.0,7.0,44.0,0.0,24.0,1
f80507,1.0,57.0,0.0,2.0,12.0,0.0,0.0,3
fa8764,0.0,60.0,1.0,7.0,30.0,0.0,19.0,1


# Extract features from the time series data itself
Import Library for Extract features

In [22]:
from seglearn.feature_functions import base_features, emg_features

from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper

c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Data Processing**

In [23]:
basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5000],
    strides=[5000],
)

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5000],
    strides=[5000],
)

fc = FeatureCollection([basic_feats, emg_feats])

In [24]:
def reader(file):
    try:
        df = pd.read_csv(file, index_col='Time', usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])

        path_split = file.split('/')
        df['Id'] = path_split[-1].split('.')[0]
        dataset = Path(file).parts[-2]
        df['Module'] = dataset
        # this is done because the speeds are at different rates for the datasets
#         if dataset == 'tdcsfog':
#             df.AccV = df.AccV / 9.80665
#             df.AccML = df.AccML / 9.80665
#             df.AccAP = df.AccAP / 9.80665

        df['Time_frac']=(df.index/df.index.max()).values#currently the index of data is actually "Time"
        
        df = pd.merge(df, tasks[['Id','t_group']], how='left', on='Id').fillna(-1)
        
        df = pd.merge(df, metadata_w_subjects[['Id','Subject', 'Visit','Test','Medication','s_group']], how='left', on='Id').fillna(-1)
        
        df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        
#         # stride
#         df["Stride"] = df["AccV"] + df["AccML"] + df["AccAP"]

#         # step
#         df["Step"] = np.sqrt(abs(df["Stride"]))
    
        df.fillna(method="ffill", inplace=True)
        
        return df
    except: pass

train = pd.concat([reader(f) for f in tqdm(train)]).fillna(0); print(train.shape)
cols = [c for c in train.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']

100%|██████████| 970/970 [01:23<00:00, 11.64it/s]


(20588374, 72)


In [25]:
train.head()

,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,Module,Time_frac,t_group,...,AccV__minimum__w=5000,AccV__mse__w=5000,AccV__root_mean_square__w=5000,AccV__skew__w=5000,AccV__slope_sign_changes__w=5000,AccV__std__w=5000,AccV__var__w=5000,AccV__waveform_length__w=5000,AccV__willison_amplitude__w=5000,AccV__zero_crossing__w=5000
0,-1.0,0.044129,-0.25,0,0,0,,defog,0.000000,-1.0,...,-1.615932,4959.956055,0.995988,-1.349459,3744.0,0.083008,0.00689,54.182549,4999.0,4999.0
1,-1.0,0.034431,-0.25,0,0,0,,defog,0.000006,-1.0,...,-1.615932,4959.956055,0.995988,-1.349459,3744.0,0.083008,0.00689,54.182549,4999.0,4999.0
2,-1.0,0.031250,-0.25,0,0,0,,defog,0.000012,-1.0,...,-1.615932,4959.956055,0.995988,-1.349459,3744.0,0.083008,0.00689,54.182549,4999.0,4999.0
3,-1.0,0.031250,-0.25,0,0,0,,defog,0.000018,-1.0,...,-1.615932,4959.956055,0.995988,-1.349459,3744.0,0.083008,0.00689,54.182549,4999.0,4999.0
4,-1.0,0.031250,-0.25,0,0,0,,defog,0.000025,-1.0,...,-1.615932,4959.956055,0.995988,-1.349459,3744.0,0.083008,0.00689,54.182549,4999.0,4999.0


In [26]:
train=train.reset_index(drop=True)

In [27]:
train.StartHesitation.value_counts()

0    20283084
1      305290
Name: StartHesitation, dtype: int64

In [28]:
train.Turn.value_counts()

0    18322106
1     2266268
Name: Turn, dtype: int64

In [29]:
train.Walking.value_counts()

0    20282017
1      306357
Name: Walking, dtype: int64

In [30]:
train.columns

Index(['AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn', 'Walking', 'Id',
       'Module', 'Time_frac', 't_group', 'Subject', 'Visit', 'Test',
       'Medication', 's_group', 'AccAP__abs_energy__w=5000',
       'AccAP__abs_sum__w=5000', 'AccAP__emg_var__w=5000',
       'AccAP__kurt__w=5000', 'AccAP__maximum__w=5000', 'AccAP__mean__w=5000',
       'AccAP__mean_abs__w=5000', 'AccAP__mean_crossings__w=5000',
       'AccAP__median__w=5000', 'AccAP__minimum__w=5000', 'AccAP__mse__w=5000',
       'AccAP__root_mean_square__w=5000', 'AccAP__skew__w=5000',
       'AccAP__slope_sign_changes__w=5000', 'AccAP__std__w=5000',
       'AccAP__var__w=5000', 'AccAP__waveform_length__w=5000',
       'AccAP__willison_amplitude__w=5000', 'AccAP__zero_crossing__w=5000',
       'AccML__abs_energy__w=5000', 'AccML__abs_sum__w=5000',
       'AccML__emg_var__w=5000', 'AccML__kurt__w=5000',
       'AccML__maximum__w=5000', 'AccML__mean__w=5000',
       'AccML__mean_abs__w=5000', 'AccML__mean_crossings__w=5000

In [31]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import average_precision_score, make_scorer

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

import xgboost as xgb

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [34]:
from sklearn.base import clone


def custom_average_precision(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return 'average_precision', score, True

def objectiveLGBM(trial: Trial, X, y):
    param = {
        'objective': 'regression',
        'metric': 'average_precision_score',
        # 'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log= True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log= True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512, step=4),
        'learning_rate': 0.01,
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0, step=0.002),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0, step=0.002),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'device':"gpu",
        'gpu_use_dp':True
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1)
        
    lgbm_regr = LGBMRegressor(**param)
    lgbm_regr = lgbm_regr.fit(X_train, y_train, verbose=False)
    
    score = average_precision_score(y_test , lgbm_regr.predict(X_test))
    return score

In [36]:
best_params_ = dict()


X_all=train.loc[:, cols].to_numpy()

for task in ['StartHesitation', 'Turn', 'Walking']:
    
    y_all = train.loc[:, task].to_numpy()

    study = optuna.create_study(direction='maximize',sampler=TPESampler())
    study.optimize(lambda trial : objectiveLGBM(trial, X_all,  y_all), n_trials=10)
    print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))
    best_params_[task] = study.best_trial.params
    

[I 2023-06-08 01:30:01,077] A new study created in memory with name: no-name-97b50a56-2c85-407d-8613-02f84e18ef79
c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [2, 512] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 510].
  warnings.warn(
c:\Users\ayun3\anaconda3\envs\test-0\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-06-08 01:40:08,676] Trial 0 finished with value: 0.9991463775864262 and parameters: {'lambda_l1': 6.079863008815342e-07, 'lambda_l2': 0.4045738422323685, 'num_leaves': 406, 'n_estimators': 1012, 'feature_fraction': 0.49, 'bagging_fraction': 0.5800000000000001, 'bagging_freq': 1, 'min_

In [ ]:
from sklearn.model_selection import GroupKFold
import pickle

N_FOLDS=5
kfold = GroupKFold(N_FOLDS)
group_var = train.Subject
groups=kfold.split(train, groups=group_var)

mAP = []
regs={}
for task in ['StartHesitation', 'Turn', 'Walking']:

    best_param = best_params_[task]
    lgbm = LGBMRegressor(**best_param, device="gpu",gpu_use_dp=True, objective='regression', metric='average_precision_score',  learning_rate= 0.01, boosting_type='gbdt')

    cvs= []
    for fold, (tr_idx,te_idx ) in enumerate(tqdm(groups, total=N_FOLDS, desc="Folds")):
        tr_idx=pd.Series(tr_idx).sample(n=2000000,random_state=42).values
        
    #     # Wrap the base regressor with the MultiOutputRegressor
    #     multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)
        x_tr,y_tr=train.loc[tr_idx,cols].to_numpy(),train.loc[tr_idx, task].to_numpy()
        x_te,y_te=train.loc[te_idx,cols].to_numpy(),train.loc[te_idx, task].to_numpy()

        # Create a base XGBoost regressor with the common parameters
        lgbm.fit(
        x_tr,y_tr,
        eval_set=(x_te,y_te),
        eval_metric=custom_average_precision,
        early_stopping_rounds=15,
        )
        
        regs[(task,fold)] = lgbm
        pickle.dump(lgbm, open(f'LGBMs/{task}_{fold}.pkl', 'wb'))

        cv=average_precision_score(y_te, lgbm.predict(x_te).clip(0.0,1.0))
        cvs.append = cv
    
    mAP.append(cvs)
    
print(cvs)
# print(np.mean(cvs))

# Build Submission File

In [ ]:
sub = pd.read_csv(path.join(root, 'sample_submission.csv'))
sub['t'] = 0
submission = []
for f in test:
    df = pd.read_csv(f)
    df.set_index('Time', drop=True, inplace=True)

    df['Id'] = f.split('/')[-1].split('.')[0]

    dataset = Path(f).parts[-2]
        
#     if dataset == 'tdcsfog':
#         df.AccV = df.AccV / 9.80665
#         df.AccML = df.AccML / 9.80665
#         df.AccAP = df.AccAP / 9.80665
            
    df['Time_frac']=(df.index/df.index.max()).values#currently the index of data is actually "Time"
    df = pd.merge(df, tasks[['Id','t_group']], how='left', on='Id').fillna(-1)

    df = pd.merge(df, metadata_w_subjects[['Id','Subject', 'Visit','Test','Medication','s_group']], how='left', on='Id').fillna(-1)
    df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
    df = df.merge(df_feats, how="left", left_index=True, right_index=True)
    df.fillna(method="ffill", inplace=True)

#     # stride
#     df["Stride"] = df["AccV"] + df["AccML"] + df["AccAP"]

#     # step
#     df["Step"] = np.sqrt(abs(df["Stride"]))
        
    res_vals=[]
    for task in ['StartHesitation', 'Turn', 'Walking']:
        for i_fold in range(N_FOLDS):
            res_val=np.round(regs[(task, i_fold)].predict(df[cols]).clip(0.0,1.0),3)
            res_vals.append(np.expand_dims(res_val,axis=2))
        res_vals=np.mean(np.concatenate(res_vals,axis=2),axis=2)
        res = pd.DataFrame(res_vals, columns=task)
        df = pd.concat([df,res], axis=1)
    
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    submission.append(df[scols])
submission = pd.concat(submission)
submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
submission[scols].to_csv('submission.csv', index=False)

In [ ]:
submission